In [1]:
# IMPORT MODULES
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras.models import Model


Using TensorFlow backend.


In [2]:
# -----------------------------1.客製化模型--------------------------------
# 載入keras模型(更換輸出圖片尺寸)
from keras.applications import DenseNet201
model = DenseNet201(include_top=False,
                 weights='imagenet',
                 input_tensor=Input(shape=(480, 360, 1))
                 )

# 定義輸出層
x = model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(360*480, activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)

# 編譯模型
model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#######################################################
# 目前只有測試到這裡!! 還有儲存模型

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
# -----------------------5.儲存模型------------------------
# save_model_name = "Densenet201_retrained_v2.h5"
save_model_name = "Densenet201_retrained_v2.pb"
# save_model_name = "Densenet201_retrained_v2.pbtxt"
model.save('./' + save_model_name)
print('已儲存'+save_model_name)

已儲存Densenet201_retrained_v2.pb


In [6]:
import tensorflow as tf
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())

    frozen_graph = tf.graph_util.convert_variables_to_constants(
        sess, tf.get_default_graph().as_graph_def(), ['conv_3/BiasAdd'])

    tf.io.write_graph(frozen_graph, "../pb/", "frozen_model.pb", as_text=False)
    tf.io.write_graph(frozen_graph, "../pb/", "frozen_model.pbtxt", as_text=True)

    tf.summary.FileWriter(OUTPUT_PATH, graph=frozen_graph, filename_suffix='_model_value')





Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


AssertionError: conv_3/BiasAdd is not in graph

In [7]:
# 把 tensorflow 的 model 存成 .onnx 的檔案
import tf2onnx

spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
output_path = "./ONNX_model/" + model.name + ".onnx"
print("output_path : ", output_path)
# model 產出在上一層的 ONNX_model 資料夾

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]

output_path :  ../ONNX_model/model_1.onnx
INFO:tensorflow:Froze 1013 variables.
INFO:tensorflow:Converted 1013 variables to const ops.



Removing optional input keras_learning_phase that appears to be a keras learning phase parameter. Use --ignore_default to force this into an input.
Removing optional input keras_learning_phase that appears to be a keras learning phase parameter. Use --ignore_default to force this into an input.
